In [ ]:
#! pip install transformers datasets evaluate

In [47]:
model_checkpoint="bert-base-uncased"
batch_size = 16
squad_v2=False

In [48]:
import datasets
from datasets import load_dataset, load_metric

In [49]:
def testing_model(context,question):
  inputs = tokenizer([context], [question], return_tensors="np")
  outputs = model(inputs)
  start_position = np.argmax(outputs.start_logits[0])
  end_position = np.argmax(outputs.end_logits[0])
  # Extract this substring from the inputs
  answer = inputs["input_ids"][0, start_position: end_position + 1]
  true_answer=tokenizer.decode(answer)
  return true_answer

In [50]:
from datasets import load_dataset

Datasets = load_dataset("squad_v2")

Reusing dataset squad_v2 (/root/.cache/huggingface/datasets/squad_v2/squad_v2/2.0.0/09187c73c1b837c95d9a249cd97c2c3f1cebada06efe667b4427714b27639b1d)


  0%|          | 0/2 [00:00<?, ?it/s]

In [51]:
Datasets

DatasetDict({
    train: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 130319
    })
    validation: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 11873
    })
})

In [52]:
train=Datasets['train'].select(range(6000))
validation=Datasets['validation'].select(range(1200))
dataset = datasets.DatasetDict({"train":train,"validation":validation})
dataset


DatasetDict({
    train: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 6000
    })
    validation: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 1200
    })
})

In [53]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [54]:
import transformers

assert isinstance(tokenizer, transformers.PreTrainedTokenizerFast)

In [55]:
tokenizer("Are you sad?", "yes Iam sad.")

{'input_ids': [101, 2024, 2017, 6517, 1029, 102, 2748, 24264, 2213, 6517, 1012, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [56]:
max_length = 384  # The maximum length of a feature (question and context)
doc_stride = 128  # The allowed overlap between two part of the context when splitting is performed.

# preparing the data

In [57]:
pad_on_right = tokenizer.padding_side == "right"

In [58]:
def prepare_train_features(examples):
    # Tokenize our examples with truncation and padding, but keep the overflows using a stride. This results
    # in one example possible giving several features when a context is long, each of those features having a
    # context that overlaps a bit the context of the previous feature.
    tokenized_examples = tokenizer(
        examples["question" if pad_on_right else "context"],
        examples["context" if pad_on_right else "question"],
        truncation="only_second" if pad_on_right else "only_first",
        max_length=max_length,
        stride=doc_stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    # Since one example might give us several features if it has a long context, we need a map from a feature to
    # its corresponding example. This key gives us just that.
    sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")
    # The offset mappings will give us a map from token to character position in the original context. This will
    # help us compute the start_positions and end_positions.
    offset_mapping = tokenized_examples.pop("offset_mapping")

    # Let's label those examples!
    tokenized_examples["start_positions"] = []
    tokenized_examples["end_positions"] = []

    for i, offsets in enumerate(offset_mapping):
        # We will label impossible answers with the index of the CLS token.
        input_ids = tokenized_examples["input_ids"][i]
        cls_index = input_ids.index(tokenizer.cls_token_id)

        # Grab the sequence corresponding to that example (to know what is the context and what is the question).
        sequence_ids = tokenized_examples.sequence_ids(i)

        # One example can give several spans, this is the index of the example containing this span of text.
        sample_index = sample_mapping[i]
        answers = examples["answers"][sample_index]
        # If no answers are given, set the cls_index as answer.
        if len(answers["answer_start"]) == 0:
            tokenized_examples["start_positions"].append(cls_index)
            tokenized_examples["end_positions"].append(cls_index)
        else:
            # Start/end character index of the answer in the text.
            start_char = answers["answer_start"][0]
            end_char = start_char + len(answers["text"][0])

            # Start token index of the current span in the text.
            token_start_index = 0
            while sequence_ids[token_start_index] != (1 if pad_on_right else 0):
                token_start_index += 1

            # End token index of the current span in the text.
            token_end_index = len(input_ids) - 1
            while sequence_ids[token_end_index] != (1 if pad_on_right else 0):
                token_end_index -= 1

            # Detect if the answer is out of the span (in which case this feature is labeled with the CLS index).
            if not (
                offsets[token_start_index][0] <= start_char
                and offsets[token_end_index][1] >= end_char
            ):
                tokenized_examples["start_positions"].append(cls_index)
                tokenized_examples["end_positions"].append(cls_index)
            else:
                # Otherwise move the token_start_index and token_end_index to the two ends of the answer.
                # Note: we could go after the last offset if the answer is the last word (edge case).
                while (
                    token_start_index < len(offsets)
                    and offsets[token_start_index][0] <= start_char
                ):
                    token_start_index += 1
                tokenized_examples["start_positions"].append(token_start_index - 1)
                while offsets[token_end_index][1] >= end_char:
                    token_end_index -= 1
                tokenized_examples["end_positions"].append(token_end_index + 1)

    return tokenized_examples

In [59]:
tokenized_datasets = dataset.map(
    prepare_train_features, batched=True, remove_columns=dataset["train"].column_names
)

Loading cached processed dataset at /root/.cache/huggingface/datasets/squad_v2/squad_v2/2.0.0/09187c73c1b837c95d9a249cd97c2c3f1cebada06efe667b4427714b27639b1d/cache-588dfe749f5ddebb.arrow


  0%|          | 0/2 [00:00<?, ?ba/s]

In [60]:
from transformers import TFAutoModelForQuestionAnswering

model = TFAutoModelForQuestionAnswering.from_pretrained(model_checkpoint)

All model checkpoint layers were used when initializing TFBertForQuestionAnswering.

Some layers of TFBertForQuestionAnswering were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['qa_outputs']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [61]:
model_name = model_checkpoint.split("/")[-1]
learning_rate = 2e-5
num_train_epochs = 7
weight_decay = 0.01

In [62]:

train_set = model.prepare_tf_dataset(
    tokenized_datasets["train"],
    shuffle=True,
    batch_size=batch_size,
)

validation_set = model.prepare_tf_dataset(
    tokenized_datasets["validation"],
    shuffle=False,
    batch_size=batch_size,
)

In [63]:
from transformers import create_optimizer

total_train_steps = len(train_set) * num_train_epochs

optimizer, schedule = create_optimizer(
    init_lr=learning_rate, num_warmup_steps=0, num_train_steps=total_train_steps
)

In [64]:
import tensorflow as tf

model.compile(optimizer=optimizer, jit_compile=True, metrics=["accuracy"])

No loss specified in compile() - the model's internal loss computation will be used as the loss. Don't panic - this is a common way to train TensorFlow models in Transformers! To disable this behaviour please pass a loss argument, or explicitly pass `loss=None` if you do not want your model to compute a loss.


In [65]:
from tensorflow.keras.callbacks import TensorBoard

tensorboard_callback = TensorBoard(log_dir="./qa_model_save/logs")

callbacks = [tensorboard_callback]

model.fit(
    train_set,
    validation_data=validation_set,
    epochs=num_train_epochs,
    callbacks=callbacks,
)

Epoch 1/7
381/381 [==============================] - 521s 1s/step - loss: 2.4605 - end_logits_accuracy: 0.4181 - start_logits_accuracy: 0.3878 - val_loss: 4.2467 - val_end_logits_accuracy: 0.2891 - val_start_logits_accuracy: 0.2809
Epoch 2/7
381/381 [==============================] - 478s 1s/step - loss: 1.1355 - end_logits_accuracy: 0.6886 - start_logits_accuracy: 0.6696 - val_loss: 4.8056 - val_end_logits_accuracy: 0.2925 - val_start_logits_accuracy: 0.2800
Epoch 3/7
381/381 [==============================] - 480s 1s/step - loss: 0.7699 - end_logits_accuracy: 0.7728 - start_logits_accuracy: 0.7534 - val_loss: 5.5143 - val_end_logits_accuracy: 0.2999 - val_start_logits_accuracy: 0.2891
Epoch 4/7
381/381 [==============================] - 480s 1s/step - loss: 0.5351 - end_logits_accuracy: 0.8422 - start_logits_accuracy: 0.8230 - val_loss: 6.2078 - val_end_logits_accuracy: 0.2958 - val_start_logits_accuracy: 0.2833
Epoch 5/7
381/381 [==============================] - 474s 1s/step - loss

In [66]:
context = """The dominant sequence transduction models are based on complex recurrent
or convolutional neural networks in an encoder-decoder configuration. The best performing
models also connect the encoder and decoder through an attention mechanism. We propose a new simple 
network architecture, the Transformer, based solely on attention mechanisms, dispensing with recurrence 
and convolutions entirely. Experiments on two machine translation tasks show these models to be superior in
quality while being more parallelizable and requiring significantly less time to train.""" 
question = "What kind of mechanisms is Transformer based on?"

inputs = tokenizer([context], [question], return_tensors="np")

outputs = model(inputs)

In [67]:
import numpy as np

start_position = np.argmax(outputs.start_logits[0])
end_position = np.argmax(outputs.end_logits[0])
print(start_position)
print(end_position)

# Extract this substring from the inputs
answer = inputs["input_ids"][0, start_position: end_position + 1]
print(answer)

46
28
[]


In [68]:
tokenizer.decode(answer)

''

# Evaluation 

In [69]:
batch = next(iter(validation_set))
output = model.predict_on_batch(batch)
output.keys()

odict_keys(['start_logits', 'end_logits'])

In [70]:
output.start_logits.shape, output.end_logits.shape

((16, 384), (16, 384))

In [71]:
import numpy as np

np.argmax(output.start_logits, -1), np.argmax(output.end_logits, -1)

(array([ 49,  37,  72,  80, 157,  17,  44,  13, 148, 202, 198,  52,  22,
         31,  32, 120]),
 array([ 49,  40,  76,  81, 158,  19,  44,  86, 153, 204, 200,  53,  26,
         33,  56,  11]))

In [72]:
n_best_size = 20

In [73]:
import numpy as np

start_logits = output.start_logits[0]
end_logits = output.end_logits[0]
# Gather the indices the best start/end logits:
start_indexes = np.argsort(start_logits)[-1 : -n_best_size - 1 : -1].tolist()
end_indexes = np.argsort(end_logits)[-1 : -n_best_size - 1 : -1].tolist()
valid_answers = []
for start_index in start_indexes:
    for end_index in end_indexes:
        if (
            start_index <= end_index
        ):  # We need to refine that test to check the answer is inside the context
            valid_answers.append(
                {
                    "score": start_logits[start_index] + end_logits[end_index],
                    "text": "",  # We need to find a way to get back the original substring corresponding to the answer in the context
                }
            )

In [74]:
def prepare_validation_features(examples):
    # Tokenize our examples with truncation and maybe padding, but keep the overflows using a stride. This results
    # in one example possible giving several features when a context is long, each of those features having a
    # context that overlaps a bit the context of the previous feature.
    tokenized_examples = tokenizer(
        examples["question" if pad_on_right else "context"],
        examples["context" if pad_on_right else "question"],
        truncation="only_second" if pad_on_right else "only_first",
        max_length=max_length,
        stride=doc_stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    # Since one example might give us several features if it has a long context, we need a map from a feature to
    # its corresponding example. This key gives us just that.
    sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")

    # We keep the example_id that gave us this feature and we will store the offset mappings.
    tokenized_examples["example_id"] = []

    for i in range(len(tokenized_examples["input_ids"])):
        # Grab the sequence corresponding to that example (to know what is the context and what is the question).
        sequence_ids = tokenized_examples.sequence_ids(i)
        context_index = 1 if pad_on_right else 0

        # One example can give several spans, this is the index of the example containing this span of text.
        sample_index = sample_mapping[i]
        tokenized_examples["example_id"].append(examples["id"][sample_index])

        # Set to None the offset_mapping that are not part of the context so it's easy to determine if a token
        # position is part of the context or not.
        tokenized_examples["offset_mapping"][i] = [
            (o if sequence_ids[k] == context_index else None)
            for k, o in enumerate(tokenized_examples["offset_mapping"][i])
        ]

    return tokenized_examples

In [75]:
validation_features = dataset["validation"].map(
    prepare_validation_features,
    batched=True,
    remove_columns=dataset["validation"].column_names,
)

Loading cached processed dataset at /root/.cache/huggingface/datasets/squad_v2/squad_v2/2.0.0/09187c73c1b837c95d9a249cd97c2c3f1cebada06efe667b4427714b27639b1d/cache-e2fd2bd2cf899bd6.arrow


In [76]:
validation_dataset = model.prepare_tf_dataset(
     validation_features,
     shuffle=False,
     batch_size=batch_size,
)

In [77]:
raw_predictions = model.predict(validation_dataset)

In [78]:
max_answer_length = 30

In [79]:
start_logits = output.start_logits[0]
end_logits = output.end_logits[0]
offset_mapping = validation_features[0]["offset_mapping"]
# The first feature comes from the first example. For the more general case, we will need to be match the example_id to
# an example index
context = dataset["validation"][0]["context"]

# Gather the indices the best start/end logits:
start_indexes = np.argsort(start_logits)[-1 : -n_best_size - 1 : -1].tolist()
end_indexes = np.argsort(end_logits)[-1 : -n_best_size - 1 : -1].tolist()
valid_answers = []
for start_index in start_indexes:
    for end_index in end_indexes:
        # Don't consider out-of-scope answers, either because the indices are out of bounds or correspond
        # to part of the input_ids that are not in the context.
        if (
            start_index >= len(offset_mapping)
            or end_index >= len(offset_mapping)
            or offset_mapping[start_index] is None
            or offset_mapping[end_index] is None
        ):
            continue
        # Don't consider answers with a length that is either < 0 or > max_answer_length.
        if end_index < start_index or end_index - start_index + 1 > max_answer_length:
            continue
        if (
            start_index <= end_index
        ):  # We need to refine that test to check the answer is inside the context
            start_char = offset_mapping[start_index][0]
            end_char = offset_mapping[end_index][1]
            valid_answers.append(
                {
                    "score": start_logits[start_index] + end_logits[end_index],
                    "text": context[start_char:end_char],
                }
            )

valid_answers = sorted(valid_answers, key=lambda x: x["score"], reverse=True)[
    :n_best_size
]
valid_answers

[{'score': 7.45901, 'text': 'France'},
 {'score': 1.9765372, 'text': 'in France'},
 {'score': 1.3125162, 'text': 'Normandy, a region in France'},
 {'score': 1.0826198, 'text': 'France.'},
 {'score': 0.63102746, 'text': 'region in France'},
 {'score': 0.096622944, 'text': 'a region in France'},
 {'score': -0.09343004,
  'text': '10th and 11th centuries gave their name to Normandy, a region in France'},
 {'score': -1.0704865,
  'text': 'Normanni) were the people who in the 10th and 11th centuries gave their name to Normandy, a region in France'},
 {'score': -1.2995858,
  'text': 'Normands; Latin: Normanni) were the people who in the 10th and 11th centuries gave their name to Normandy, a region in France'},
 {'score': -1.3511014,
  'text': 'the 10th and 11th centuries gave their name to Normandy, a region in France'},
 {'score': -1.4678335,
  'text': 'French: Normands; Latin: Normanni) were the people who in the 10th and 11th centuries gave their name to Normandy, a region in France'},
 {

In [80]:
dataset["validation"][0]["answers"]

{'answer_start': [159, 159, 159, 159],
 'text': ['France', 'France', 'France', 'France']}

In [81]:
import collections

examples = dataset["validation"]
features = validation_features

example_id_to_index = {k: i for i, k in enumerate(examples["id"])}
features_per_example = collections.defaultdict(list)
for i, feature in enumerate(features):
    features_per_example[example_id_to_index[feature["example_id"]]].append(i)

In [82]:
from tqdm.auto import tqdm

def postprocess_qa_predictions(
    examples,
    features,
    all_start_logits,
    all_end_logits,
    n_best_size=20,
    max_answer_length=30,
):
    # Build a map example to its corresponding features.
    example_id_to_index = {k: i for i, k in enumerate(examples["id"])}
    features_per_example = collections.defaultdict(list)
    for i, feature in enumerate(features):
        features_per_example[example_id_to_index[feature["example_id"]]].append(i)

    # The dictionaries we have to fill.
    predictions = collections.OrderedDict()

    # Logging.
    print(
        f"Post-processing {len(examples)} example predictions split into {len(features)} features."
    )

    # Let's loop over all the examples!
    for example_index, example in enumerate(tqdm(examples)):
        # Those are the indices of the features associated to the current example.
        feature_indices = features_per_example[example_index]

        min_null_score = None  # Only used if squad_v2 is True.
        valid_answers = []

        context = example["context"]
        # Looping through all the features associated to the current example.
        for feature_index in feature_indices:
            # We grab the predictions of the model for this feature.
            start_logits = all_start_logits[feature_index]
            end_logits = all_end_logits[feature_index]
            # This is what will allow us to map some the positions in our logits to span of texts in the original
            # context.
            offset_mapping = features[feature_index]["offset_mapping"]

            # Update minimum null prediction.
            cls_index = features[feature_index]["input_ids"].index(
                tokenizer.cls_token_id
            )
            feature_null_score = start_logits[cls_index] + end_logits[cls_index]
            if min_null_score is None or min_null_score < feature_null_score:
                min_null_score = feature_null_score

            # Go through all possibilities for the `n_best_size` greater start and end logits.
            start_indexes = np.argsort(start_logits)[
                -1 : -n_best_size - 1 : -1
            ].tolist()
            end_indexes = np.argsort(end_logits)[-1 : -n_best_size - 1 : -1].tolist()
            for start_index in start_indexes:
                for end_index in end_indexes:
                    # Don't consider out-of-scope answers, either because the indices are out of bounds or correspond
                    # to part of the input_ids that are not in the context.
                    if (
                        start_index >= len(offset_mapping)
                        or end_index >= len(offset_mapping)
                        or not offset_mapping[start_index]
                        or not offset_mapping[end_index]
                    ):
                        continue
                    # Don't consider answers with a length that is either < 0 or > max_answer_length.
                    if (
                        end_index < start_index
                        or end_index - start_index + 1 > max_answer_length
                    ):
                        continue
                    start_char = offset_mapping[start_index][0]
                    end_char = offset_mapping[end_index][1]
                    valid_answers.append(
                        {
                            "score": start_logits[start_index] + end_logits[end_index],
                            "text": context[start_char:end_char],
                        }
                    )

        if len(valid_answers) > 0:
            best_answer = sorted(valid_answers, key=lambda x: x["score"], reverse=True)[
                0
            ]
        else:
            # In the very rare edge case we have not a single non-null prediction, we create a fake prediction to avoid
            # failure.
            best_answer = {"text": "", "score": 0.0}

        # Let's pick our final answer: the best one or the null answer (only for squad_v2)
        if not squad_v2:
            predictions[example["id"]] = best_answer["text"]
        else:
            answer = (
                best_answer["text"] if best_answer["score"] > min_null_score else ""
            )
            predictions[example["id"]] = answer

    return predictions

In [83]:
final_predictions = postprocess_qa_predictions(
    dataset["validation"],
    validation_features,
    raw_predictions["start_logits"],
    raw_predictions["end_logits"],
)

Post-processing 1200 example predictions split into 1207 features.


  0%|          | 0/1200 [00:00<?, ?it/s]

In [84]:
final_predictions

OrderedDict([('56ddde6b9a695914005b9628', 'France'),
             ('56ddde6b9a695914005b9629', '10th and 11th centuries'),
             ('56ddde6b9a695914005b962a', 'Denmark, Iceland and Norway'),
             ('56ddde6b9a695914005b962b', 'Rollo'),
             ('56ddde6b9a695914005b962c', '10th century'),
             ('5ad39d53604f3c001a3fe8d1', 'The Normans'),
             ('5ad39d53604f3c001a3fe8d2', 'Normandy'),
             ('5ad39d53604f3c001a3fe8d3', 'Rollo'),
             ('5ad39d53604f3c001a3fe8d4', 'first half of the 10th century'),
             ('56dddf4066d3e219004dad5f', 'William the Conqueror'),
             ('56dddf4066d3e219004dad60', 'William the Conqueror'),
             ('56dddf4066d3e219004dad61', 'Catholic orthodoxy'),
             ('5ad3a266604f3c001a3fea27', 'political, cultural and military'),
             ('5ad3a266604f3c001a3fea28', 'The Normans'),
             ('5ad3a266604f3c001a3fea29',
              'The Normans were famed for their martial spirit and eve

In [85]:
metric = load_metric("squad_v2" if squad_v2 else "squad")

In [86]:
dataset["validation"]

Dataset({
    features: ['id', 'title', 'context', 'question', 'answers'],
    num_rows: 1200
})

In [87]:
dataset["validation"][0]

{'answers': {'answer_start': [159, 159, 159, 159],
  'text': ['France', 'France', 'France', 'France']},
 'context': 'The Normans (Norman: Nourmands; French: Normands; Latin: Normanni) were the people who in the 10th and 11th centuries gave their name to Normandy, a region in France. They were descended from Norse ("Norman" comes from "Norseman") raiders and pirates from Denmark, Iceland and Norway who, under their leader Rollo, agreed to swear fealty to King Charles III of West Francia. Through generations of assimilation and mixing with the native Frankish and Roman-Gaulish populations, their descendants would gradually merge with the Carolingian-based cultures of West Francia. The distinct cultural and ethnic identity of the Normans emerged initially in the first half of the 10th century, and it continued to evolve over the succeeding centuries.',
 'id': '56ddde6b9a695914005b9628',
 'question': 'In what country is Normandy located?',
 'title': 'Normans'}

In [88]:

predictions = [
    {"prediction_text": v,"id": k, "no_answer_probability": 0.0}
    for k, v in final_predictions.items()
]

references = [
    {'answers': {'answer_start': ex['answers']['answer_start'], 'text': ex['answers']['text']}, 'id': ex['id']} for ex in dataset["validation"]
]
from evaluate import load
squad_v2_metric = load("squad_v2")
results = squad_v2_metric.compute(predictions=predictions, references=references)
results

{'HasAns_exact': 60.96345514950166,
 'HasAns_f1': 72.12955115612286,
 'HasAns_total': 602,
 'NoAns_exact': 0.0,
 'NoAns_f1': 0.0,
 'NoAns_total': 598,
 'best_exact': 50.25,
 'best_exact_thresh': 0.0,
 'best_f1': 50.25,
 'best_f1_thresh': 0.0,
 'exact': 30.583333333333332,
 'f1': 36.184991496654966,
 'total': 1200}

# live Demo

In [89]:
#Put the contex here
context="""At one of the interview I got this question, Write as many test cases
 as you can for this scenario – If you are a new customer and you want to open a credit 
 card account then there are three conditions first you will get a 15% discount on all your purchases today,
  second if you are an existing customer and you hold a loyalty card, you get a 10% discount and third if you have a coupon,
   you can get 20% off today (but it can’t be used with the ‘new customer’ discount). Discount amounts are added, if applicable.

Can somebody please help me with it."""
#Put the question here
question="what was the question that he got asked ? "

In [90]:
answer=testing_model(context,question)

In [91]:
answer

'[CLS] at one of the interview i got this question, write as many test cases as you can for this scenario – if you are a new customer and you want to open a credit card account then there are three conditions first you will get a 15 % discount on all your purchases today, second if you are an existing customer and you hold a loyalty card, you get a 10 %'